La tabla Abstracts puesta a disposici ́on en el campus virtual tiene 2 colum-
nas: department, y abstract. La columna abstract contiene el resumen de una
publicaci ́on cient ́ıfica y la columna department indica un departamento univer-
sitario asociado a esa publicaci ́on (con tres valores posibles).
Las separaciones aleatorias de datos de las preguntas 1 y 2 deberan realizarse
utilizando la semilla asignada 70.

**Pregunta 1 (2 pts)**
Preparar los datos para ajustar un Modelo de Clasificaci ́on. Para esto:
1. Generar un DataFrame de Pandas para el conjunto de entrenamiento y
otro para el conjunto de test (el conjunto test debe contener el 20% de los
datos, con una distribución de clases identica a la original).

2. Implementar una función de preprocesamiento y aplicarla sobre el conjunto de datos de entrenamiento. Se deben incluir al menos los siguientes
métodos de limpieza en la implementaci ́on: 
* (i) Eliminar los caracteres que no sean letras del abecedario. 
* (ii) Estandarizar la escritura a letras minusculas. 
* (iii) Eliminar las t ́ıldes y los espacios en blanco sobrantes.
* (iv) Eliminar las stopwords.

3. Vectorizar los documentos utilizando un modelo Word2Vec preentrenado
de la librer ́ıa gensim. Indicar el modelo utilizado.

4. Construir el tensor X llevando todos los documentos a un mismo largo
(par ́ametro ajustable). Adem ́as, construir el vector y con la clasificaci ́on
de cada documento incluido en X.

In [9]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk

df=pd.read_csv(r'Abstracts.csv',header=0)
df

,department,abstract
0,1,Chilean primary healthcare practice is analyz...
1,1,Attempting to deepen the understanding of fact...
2,1,The export of fresh blueberries is an importan...
3,1,The initial microflora of minimally processed ...
4,1,"In the context of multidimensional structures,..."
...,...,...
194,3,BackgroundLupin is a protein-rich legume with ...
195,3,"Phycobiliproteins, the main polypeptidic compo..."
196,3,"There is great interest in the food, cosmetic ..."
197,3,The use of vegetable proteins as food ingredie...


In [10]:
# Dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

semilla = 70
df_train, df_test = train_test_split(df, test_size=0.2, random_state=semilla, stratify=df['department'])

In [11]:
import re
def preprocesador(text):
    puntuacion = r'[,;.:•“”¡!¿?<>@#$%&[\](){}<>~=+\-*/|\\_^`"\']' #caracteres que no son abecedario
    text = re.sub(puntuacion, ' ', text)
    text = re.sub('[\W]+', ' ', text.lower()) #convierte a minuscula
    text = re.sub(r'\d','', text)
    text = text.strip()
    return text

df_train['abstract'] = df_train['abstract'].apply(preprocesador)
df_test['abstract'] = df_test['abstract'].apply(preprocesador)

In [12]:
corpus = df['abstract'].values.tolist()
stop_words = nltk.corpus.stopwords.words('english') #elimina las stopwords
tok_corp = [nltk.word_tokenize(sent) for sent in corpus]
tok_corp = [x for x in tok_corp if x not in stop_words]

In [13]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(tok_corp, vector_size=100, window=3, min_count=1, workers=8, sg=1, alpha=0.001, epochs=10)
model.save('Word2VecSG')

In [14]:
import gensim

model = gensim.models.Word2Vec.load(r'Word2VecSG')
word_vectors = model.wv
words = list(model.wv.key_to_index)
len(words)

5660

In [15]:
import gensim.downloader 

#modelo mlp para hacer la vectorización
wv = gensim.downloader.load('glove-wiki-gigaword-50')

In [20]:

def tokenizar_y_vectorizar(dataset):
    corpus = dataset['abstract'].values.tolist()
    tok_corp = [nltk.word_tokenize(sent) for sent in corpus]
    vectorized_data = []
    for sample in tok_corp:
        vectors = [wv[w] for w in sample if w in wv.key_to_index]
        vectorized_data.append(vectors)
    return vectorized_data
X = tokenizar_y_vectorizar(df)
y = df.tópico.values


AttributeError: 'DataFrame' object has no attribute 'tópico'

Pregunta 2 (2 pts)
Ajustar un modelo para Clasificar los Abstracts. Para esto:
1. Separar el conjunto (X, y) de manera estratificada en un conjunto de en-
trenamiento, con el 70% de los datos, y un conjunto de validaci ́on con los
datos restantes. ¿Cu ́al es la dimensi ́on de los datos de entrada?. Interpre-
tar los valores de la dimensi ́on.
2. Entrenar una Red RNN para predecir el departamento asociado a cada
abstract, considerando los conjuntos generados en el item anterior.
3. Mostrar las curvas de aprendizaje del modelo durante el entrenamiento
(para los conjuntos de entrenamiento y de validaci ́on), considerando la
Funci ́on de P ́erdida y la Accuracy. Indicando si hay evidencia de sobrea-
juste o bajoajuste.

Pregunta 3 (2 ptos)
Realizar una evaluaci ́on del modelo sobre datos desconocidos. Para esto:
1. Generar un tensor llamado X test a partir de los datos del DataFrame
que contiene al conjunto de test. Aplicar el mismo m ́etodo de preproce-
samiento aplicado al conjunto de entrenamiento y utilizar el mismo modelo
preentrenado Word2Vec y el mismo largo de los documentos.
2. Utilizar el modelo entrenado en la pregunta 2 para predecir la clasificaci ́on
de todos los datos incluidos en el tensor X test.
3. Generar dos gr ́aficos de tortas: el primero para las clases reales y el se-
gundo para las clases predichas por el modelo. Cada gr ́afico de torta debe
mostrar las de clases asignando un color para cada clase. Agregar en cada
gr ́afico la cantidad de datos de cada clase. Finalmente indicar la Accuracy
obtenida por el modelo sobre los datos del conjunto test.

regunta adicional (3 pts)
1. Explorar las tareas asignadas hasta la pregunta 2 con varias configura-
ciones diferentes de los par ́ametros de dise ̃no, a saber: largo de los doc-
umentos, par ́ametros de entrenamiento (learning rate, batch, epocas) y
par ́ametros de dise ̃no de la red (n ́umero de capas, tama ̃no de las capas
intermedias, etc.). Considerar al menos 5 configuraciones diferentes de los
par ́ametros.
2. Determinar una combinaci ́on de par ́ametros, entre las configuraciones ex-
ploradas, que genere los mejores resultados para la pregunta 2. Evidenciar
esa comparaci ́on realizada.
2
3. Utilizar en la pregunta 3 la combinaci ́on optimal de par ́ametros encon-
trada.